# Topic Modeling

### Author: [Marco Tavora](http://www.marcotavora.me/)

**For the best viewing experience use [nbviewer]().**

## Table of contents

- [Introduction](#Introduction)

- [Required libraries](#Required-libraries)

## Introduction

[[go back to the top]](#Table-of-contents)

In this notebook, I will use Python and its libraries for **topic modeling**. In topic modeling, statistical models are used to identify topics or categories in a document or a set of documents. I will use one specific method called **Latent Dirichlet Allocation (LDA)**. The algorithm can be summarized as follows:
- First we select - without previous knowledge regarding what the topics actually are - a fixed number of topics $T$ 
- We then randomly assign each word to a topic
- For each document $d$, word $w$ and topic $t$ we calculate the probability $P(t\,|\,w,d)$ that the word $w$ of document $d$ corresponds to topic $t$
- We then reassign each word $w$ to some topic based on $P(t\,|\,w,d)$ and repeat the process until we find the optimal assignment of words to topics

## Libraries  

[[go back to the top]](#Table-of-contents)

This notebook uses the following packages:

- `spacy`
- `nltk`
- `random`
- `gensim`
- `pickle`
- `pandas`
- `sklearn`

In [1]:
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # see the value of multiple statements at once.
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## The problem domain

[[go back to the top]](#Table-of-contents)

In this project I apply LDA to labels on research papers. The dataset is a subset of [this](https://github.com/susanli2016/Machine-Learning-with-Python/blob/master/dataset.csv) data set.

## Using `spaCy`

[[go back to the top]](#Table-of-contents)

In this projects I will use the `spaCy` library (see this [link](https://github.com/skipgram/modern-nlp-in python/blob/master/executable/Modern_NLP_in_Python.ipynb)). 

`spaCy` is:

> An industrial-strength natural language processing (NLP) library for Python. spaCy's goal is to take recent advancements in natural language processing out of research papers and put them in the hands of users to build production software.

In [2]:
import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()

## Importing the documents

[[go back to the top]](#Table-of-contents)

In [3]:
df = pd.read_csv('articles.csv',header=None)
df.columns = ['titles']
df.shape
df.head()

(100, 1)

,titles
0,A novel digitally controlled low noise ring os...
1,A motion compensation system with a high effic...
2,A reconfigurable MAC architecture implemented ...
3,Why is 3-D interaction so hard and what can we...
4,Automatic colorization of grayscale images usi...


## List of documents

[[go back to the top]](#Table-of-contents)

From `df` I will build a list `doc_set` containing the row entries:

In [4]:
doc_set = df.values.T.tolist()[0]
print(doc_set[0:10])

['A novel digitally controlled low noise ring oscillator.', 'A motion compensation system with a high efficiency reference frame pre-fetch scheme for QFHD H.264/AVC decoding.', 'A reconfigurable MAC architecture implemented with mixed-Vt standard cell library.', 'Why is 3-D interaction so hard and what can we really do about it?', 'Automatic colorization of grayscale images using multiple images on the web.', 'Automatic Profile Generation in eRACE.', 'A QoS aware multicore hash scheduler for network applications.', 'Incremental Bloom Filters.', 'Hardware Organization for Nonnumeric Processing', 'Retrieval of motion capture data based on short-term feature extraction.']


## Cleaning the text

[[go back to the top]](#Table-of-contents)

Before applying natural language processing tools to our problem, I will provide a quick review of some basic procedures using Python. We first import `nltk` and the necessary classes for lemmatization and stemming:

In [5]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

We then create objects of the classes `PorterStemmer` and `WordNetLemmatizer`:

In [6]:
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

To use lemmatization and/or stemming in a given string text we must first tokenize it. The code below matches word characters until it reaches a non-word character, like a space. 

In [7]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

### Build a list of lists of tokens 

In [8]:
tokenined_docs = []
for doc in doc_set:
    tokens = tokenizer.tokenize(doc.lower())
    tokenined_docs.append(tokens)
    
print(tokenined_docs[0:3])

[['a', 'novel', 'digitally', 'controlled', 'low', 'noise', 'ring', 'oscillator'], ['a', 'motion', 'compensation', 'system', 'with', 'a', 'high', 'efficiency', 'reference', 'frame', 'pre', 'fetch', 'scheme', 'for', 'qfhd', 'h', '264', 'avc', 'decoding'], ['a', 'reconfigurable', 'mac', 'architecture', 'implemented', 'with', 'mixed', 'vt', 'standard', 'cell', 'library']]


### Apply lemmatizing

In [9]:
lemmatized_tokens = []
for lst in tokenined_docs:
    tokens_lemma = [lemmatizer.lemmatize(i) for i in lst]
    lemmatized_tokens.append(tokens_lemma)
    
print(lemmatized_tokens[0:3])

[['a', 'novel', 'digitally', 'controlled', 'low', 'noise', 'ring', 'oscillator'], ['a', 'motion', 'compensation', 'system', 'with', 'a', 'high', 'efficiency', 'reference', 'frame', 'pre', 'fetch', 'scheme', 'for', 'qfhd', 'h', '264', 'avc', 'decoding'], ['a', 'reconfigurable', 'mac', 'architecture', 'implemented', 'with', 'mixed', 'vt', 'standard', 'cell', 'library']]


### Dropping stopwords and words with less than $n$ letters

In [10]:
from stop_words import get_stop_words
en_stop_words = get_stop_words('en')

In [11]:
n=4
tokens = []
for lst in lemmatized_tokens:
    tokens.append([i for i in lst if not i in en_stop_words if len(i) > n])

print(tokens[0:3])

[['novel', 'digitally', 'controlled', 'noise', 'oscillator'], ['motion', 'compensation', 'system', 'efficiency', 'reference', 'frame', 'fetch', 'scheme', 'decoding'], ['reconfigurable', 'architecture', 'implemented', 'mixed', 'standard', 'library']]


## Document-term matrix

[[go back to the top]](#Table-of-contents)

I will now generate an LDA model and for that, the frequency that each term occurs within each document needs to be understood.

A **document-term matrix** is constructed to do that. It contains a corpus of $n$ documents and a vocabulary of $m$ words. Each cell $ij$ counts the frequency of the word $j$ in the document $i$.

|               | word_1 | word_2 | ... | word_m |
| ------------- |:------:| ----- :|----- :|----- :|
| doc_1         | 1      | 3   | ... |2
| doc_2         | 2      |   3   |...|3
| ...           | ...    |    2   |...|1
| doc_n         | 1      |    1   |...|1

What LDA does is to convert this matrix into two matrices with lower dimensions namely:

|               | topic_1 | topic_2 | ... | topic_T |
| ------------- |:------:| ----- :|----- :|----- :|
| doc_1         | 0      | 1   | ... |1
| doc_2         | 0      |   1   |...|1
| ...           | ...    |    ...   |...|1
| doc_n         | 1      |    0   |...|0

and

|               | word_1 | word_2 | ... | word_m |
| ------------- |:------:| ----- :|----- :|----- :|
| topic_1         | 1      | 0   | ... |1
| topic_2         | 1      |   0   |...|1
| ...           | ...    |    ...   |...|1
| topic_T         | 1      |    1   |...|1




## Tokens into dictionary

[[go back to the top]](#Table-of-contents)

In [12]:
from gensim import corpora, models

dictionary = corpora.Dictionary(tokens)

## Tokenize documents into document-term matrix

[[go back to the top]](#Table-of-contents)

In [32]:
corpus = [dictionary.doc2bow(text) for text in tokens]

import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [16]:
corpus[0]

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]

## LDA model

In [28]:
import gensim
ldamodel_3 = gensim.models.ldamodel.LdaModel(corpus, num_topics=3, id2word = dictionary, passes=20)
ldamodel_3.save('model3.gensim')
ldamodel_4 = gensim.models.ldamodel.LdaModel(corpus, num_topics=3, id2word = dictionary, passes=20)
ldamodel_4.save('model4.gensim')

In [29]:
for el in ldamodel_3.print_topics(num_topics=3, num_words=3):
    print(el,'\n')

(0, '0.017*"system" + 0.017*"image" + 0.017*"based"') 

(1, '0.035*"network" + 0.015*"multi" + 0.012*"based"') 

(2, '0.016*"based" + 0.013*"using" + 0.012*"system"') 



In [30]:
for el in ldamodel_4.print_topics(num_topics=3, num_words=3):
    print(el,'\n')

(0, '0.028*"based" + 0.023*"system" + 0.015*"database"') 

(1, '0.015*"search" + 0.012*"multi" + 0.012*"analysis"') 

(2, '0.025*"network" + 0.016*"image" + 0.013*"using"') 



In [33]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')

In [34]:
corpus = pickle.load(open('corpus.pkl', 'rb'))

In [35]:
lda = gensim.models.ldamodel.LdaModel.load('model3.gensim')

In [36]:
import pyLDAvis.gensim

In [37]:
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)

/Users/marcotavora/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


In [38]:
pyLDAvis.display(lda_display)